In [3]:
import json

from datetime import datetime, timedelta, timezone, time, date

import pandas as pd
import numpy as np

In [4]:
def get_arrivals(filename = 'sample_routes_stops_15s.json'):
    with open(filename, 'r') as f:
        return json.load(f)

In [10]:
arrivals = pd.DataFrame.from_dict(get_arrivals())

In [15]:
arrivals.sample(10)

,index,DID,ROUTE,SID,TIME,VID
15805,15805,14___O_F00,14,5539,1539623292000,7234
15405,15405,14___O_F00,14,5532,1539652851000,7261
14784,14784,1____I_F00,1,3893,1539636256000,5624
15572,15572,14___O_F00,14,5536,1539616659000,7290
13809,13809,1____I_F00,1,3825,1539637816000,5603
16387,16387,14___O_F00,14,5548,1539617500000,7214
4220,4220,1____O_F00,1,3892,1539627328000,5595
15459,15459,14___O_F00,14,5534,1539656408000,7249
20403,20403,14___I_F00,14,5582,1539603830000,7244
13915,13915,1____I_F00,1,3823,1539663235000,5626


In [16]:
# find the smallest nonnegative waiting time
def absmin(series):
    return series[series >= 0].min()


# # input: df with entries from one day
# # possible optimzation: sort df by timestamp, then pick first timestamp > minute for each minute (need to time to make sure but should be faster)
def minimum_waiting_times(df, start_time, end_time, group):
    minute_range = [start_time + timedelta(minutes=i) for i in range(
        (end_time - start_time).seconds//60)]
    wait_times = pd.DataFrame(columns=[])

    for minute in minute_range:
        # TODO (jtanquil): we get this error, see if you can fix it
        # A value is trying to be set on a copy of a slice from a DataFrame.
        # Try using .loc[row_indexer,col_indexer] = value instead
        # See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
        #   df['WAIT'] = df['timestamp'].apply(lambda x: (x - minute).total_seconds())
        df['WAIT'] = df['timestamp'].apply(lambda x: (x - minute).total_seconds())
        pivot = df[group + ['WAIT']].pivot_table(values = ['WAIT'], index = group, aggfunc = absmin)
        pivot['TIME'] = minute
        pivot = pivot.reset_index()
        wait_times = wait_times.append(pivot, sort = True)

    return wait_times

def all_wait_times(df, timespan = ("00:00", "23:59"), group = []):
    # parse timestamp
    df[
    dates = df['DATE'].unique()
    avg_over_pd = pd.DataFrame(columns = group + ['DATE', 'TIME', 'WAIT'])

    for date in dates:
        #print(f"{datetime.now().strftime('%a %b %d %I:%M:%S %p')}: start processing {date}.")
        start_time = datetime.strptime(f"{date.isoformat()} {timespan[0]} -0800", "%Y-%m-%d %H:%M %z")
        end_time   = datetime.strptime(f"{date.isoformat()} {timespan[1]} -0800", "%Y-%m-%d %H:%M %z")
        daily_wait = minimum_waiting_times(df[df['DATE'] == date], start_time, end_time, group)
        #print(f"{datetime.now().strftime('%a %b %d %I:%M:%S %p')}: found waits for {date}.")
        #daily_wait = daily_wait.pivot_table(values = ['WAIT'], index = group).reset_index()
        daily_wait['DATE'] = date
        daily_wait['TIME'] = daily_wait['TIME'].apply(lambda x: x.time())
        avg_over_pd = avg_over_pd.append(daily_wait, sort = True)

    return avg_over_pd

def quantiles(series):
    return [np.percentile(series, i) for i in [5, 25, 50, 75, 95]]

def get_summary_statistics(df, group):
    waits = df.pivot_table(values = ['WAIT'], index = group, aggfunc = {'WAIT': [np.mean, np.std, quantiles]}).reset_index()
    waits.columns = ['_'.join(col) if col[0] == 'WAIT' else ''.join(col) for col in waits.columns.values]
    waits[[f"{i}th percentile" for i in [5, 25, 50, 75, 95]]] = waits['WAIT_quantiles'].apply(lambda x: pd.Series(x))
    waits = waits.drop('WAIT_quantiles', axis = 1)
    return waits

In [ ]:
waits = all_wait_times(arrivals)

In [19]:
arrivals['TIME'].apply(lambda x: datetime.fromtimestamp(x))

OSError: [Errno 22] Invalid argument

In [20]:
datetime.fromtimestamp(1539623292000)

OSError: [Errno 22] Invalid argument